In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
import math
# from perception import find_ball
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
#RealWorld.addFile("../../scenarios/panda_moveGripper.g")
RealWorld.addFile("../../scenarios/setup.g")

# Workaround, need ot fix later
#q = RealWorld.getJointState()
#q[7] = math.pi
#RealWorld.setJointState(q)

V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
# Sample target Object
#radius = 0.04

#targetObj=RealWorld.addFrame("obj0")
#targetObj.setColor([1.,1.,0.])
#targetObj.setShape(ry.ST.sphere, [radius])
##targetObj.setShape(ry.ST.capsule, [0.05, 0.05])
##targetObj.setPosition([-0.1, .3, 1.25])
#targetObj.setPosition([-0.2, .3, 1.25])
#targetObj.setMass(0.1)
#targetObj.setContact(1)


In [4]:
V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

# instantiate the simulation
#S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
#print(S)

In [5]:
from environment import *
from ballmanagement import *
ball_management = BallManagement(RealWorld, ry, math)
ball = ball_management.create_ball("ball", 2.4, 0.5, 0.1)
ball_management.create_ball("traject", -1.9, 1, 0.04)
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
environment = Environment(S, V, RealWorld, ry, time, np, math)

In [6]:
V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

In [7]:
thrower_state = 1

while True:
    
    # Control flow for the thrower 
    if thrower_state == 1 or thrower_state == 2:
        environment.thrower_set_move_to_objective(ball.get_position())
        if thrower_state == 1 and environment.thrower_is_move_to_objective_fulfilled():
            thrower_state = 2
            print("Done moving")
            environment.thrower_set_grab_objective(True)
        if thrower_state == 2 and environment.thrower_is_gripper_grasping():
            thrower_state = 3
            print("Done grasping")
            
    if thrower_state == 3:
        pos = ball.get_position()
        environment.thrower_set_move_to_objective([2.6, .5, .1])
        if environment.thrower_is_move_to_objective_fulfilled():
            thrower_state = 4
            print("Done lifting.")
            environment.thrower_set_throw_objective([-1, 0])
            
    if thrower_state == 4:
        if not environment.thrower_is_gripper_grasping():
            thrower_state = 5
            print("Done throwing.")
 
    if thrower_state == 5:
        previous_pos = ball.get_position()
        time.sleep(.01)
        S.step([], .01, ry.ControlMode.none)
        ball.set_velocity(.01, previous_pos)
        traject = ball.get_trajectory_estimate([-2, 0, 0, 1])
        print(traject)
        RealWorld.getFrame("traject").setPosition(traject)
        V.recopyMeshes(RealWorld)
        V.setConfiguration(RealWorld)
        
    # Control flow for Goalee
        
    environment.execute_time_step(.01)
    


Done moving
Done grasping
Done lifting.
Overriding the move_to and grab objectives...
Done throwing.
traject_pos:
[1.918497920036316, 0.0011059922398999333]
traject_dir:
[-5.482270038973736, 0.03925099329441414]
plane_pos:
[-2, 0]
plane_dir:
[0, 1]
xx:
0.7147582830067682
[-2.0, 0.029160964813325557, 0.7632068720033861]
traject_pos:
[1.6793690919876099, 0.002831690013408661]
traject_dir:
[-5.380090131471371, 0.038825976203882395]
plane_pos:
[-2, 0]
plane_dir:
[0, 1]
xx:
0.6838861435545058
[-2.0, 0.029384237149220803, 0.8199142839328096]
traject_pos:
[1.4402402639389038, 0.00455738790333271]
traject_dir:
[-5.291067790840795, 0.03818353728660139]
plane_pos:
[-2, 0]
plane_dir:
[0, 1]
xx:
0.6501977294439882
[-1.9999999999999996, 0.0293842371492208, 0.8774111592914423]
traject_pos:
[1.2011114358901978, 0.00628308579325676]
traject_dir:
[-5.198455417519112, 0.0375151905275501]
plane_pos:
[-2, 0]
plane_dir:
[0, 1]
xx:
0.6157812616998228
[-2.0, 0.0293842371492208, 0.9309764444868558]
traject_po

TypeError: setPosition(): incompatible function arguments. The following argument types are supported:
    1. (self: libry.Frame, arg0: List[float]) -> None

Invoked with: <libry.Frame object at 0x7fb78bf38ab0>, None

In [ ]:
# Frame details print

print("FrameNames : \n", RealWorld.getFrameNames(), "\n");
#print("JointNames : \n",C.getJointNames(), "\n");
#print("getJointDimension : \n",C.getJointDimension(), "\n");
print("GetJointState old : \n", RealWorld.getJointState(), "\n");
#print (RealWorld.frame("obj0").getPosition())

#[y,J] = RealWorld.evalFeature(ry.FS.vectorZ, ["paddle"])
#print (y)

#pi = math.pi
#
#q = RealWorld.getJointState()
#q[0] = pi
#q[1] = 0
#q[2] = 1
#q[3] = 0
#
#RealWorld.setJointState(q)
#
#print("GetJointState new : \n", RealWorld.getJointState(), "\n");
#
## updating the display
#V.recopyMeshes(RealWorld)
#V.setConfiguration(RealWorld)

In [ ]:
def grab():
    
    tau = .01
    close = False
    
    print("-------- Grabbing -----------")

    while True:
        if S.getGripperIsGrasping("gripper"): break
        
        time.sleep(tau)
        #grab sensor readings from the simulation
        q = S.get_q()
        
        if not close:
            [y, J] = RealWorld.evalFeature( ry.FS.positionDiff, ["gripperCenter", "obj0"])
            error = np.abs(y).max()
            #print(error)
                                   
            if error < 2.5e-2:
                close = True
                print("Closing Gripper")
                S. closeGripper ("gripper")
                print( "------ Closed Gripper -------")
                
        komo = RealWorld.komo_path(1.,1, tau, True)
        komo. clearObjectives()

        # For smooth q                       
        komo.add_qControlObjective( order=1, scale=1e0)
        # To prevent collisions
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e4])
        # Position difference between gripper centre is 0
        komo.addObjective([1.], ry.FS.positionDiff, ['gripperCenter', 'a'], ry.OT.sos, [1e1])
        # X axis is parallel
        komo.addObjective([], ry.FS.vectorX, ["gripperCenter"], ry.OT.eq, [1e1], target = [1,0.,0])
        # Y axis is parallel
        komo.addObjective([], ry.FS.vectorY, ["gripperCenter"], ry.OT.eq, [1e1], target = [0.,1,0])
        # To keep gripper open, 
        komo.addObjective([], ry.FS.qItself, ['finger1'], ry.OT.eq, scale=[1e1], order=1)

        komo.optimize()
        
        RealWorld.setFrameState(komo.getConfiguration(0))
        # Get joint state
        q = RealWorld.getJointState()

        # updating the display
        V.recopyMeshes(RealWorld)
        V.setConfiguration(RealWorld)
        
        S.step(q, tau, ry.ControlMode.position)

In [ ]:
def wait(timesteps):
    tau = .01

    for t in range(timesteps):
        time.sleep(tau)
        
        S.step([], tau, ry.ControlMode.none)

In [ ]:
def liftandthrow():
    throwpoint = 1
    tau = 0.01
    flag = 0
    
    print("------- Lift and Throw ------")
    
    while throwpoint >= 0.5e-2:
        
        [pos, Jpos] = RealWorld.evalFeature( ry.FS.position, ["gripperCenter"])
        throwpoint = np.linalg.norm(pos-[0.25,0.25,1])
        #print(throwpoint)
            
        komo = RealWorld.komo_path(1.,1, tau, True)
        komo. clearObjectives()

        # For smooth q                       
        komo.add_qControlObjective( order=1, scale=0.8)
        # To prevent collisions
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e4])
        # Position difference between gripper centre is 0
        komo.addObjective([1.], ry.FS.position, ['gripperCenter'], ry.OT.sos, [1e1], target=[0.25,0.25,1])
        # To keep gripper closed, 
        komo.addObjective([], ry.FS.qItself, ['finger1'], ry.OT.eq, scale=[1e1], order=1)

        if throwpoint <= 0.3 and flag == 0:
            flag = 1
            S.openGripper ("gripper")
            print( "------ Open Gripper -------")    
            
        komo.optimize()
        
        RealWorld.setFrameState(komo.getConfiguration(0))
        # Get joint state
        q = RealWorld.getJointState()

        # updating the display
        V.recopyMeshes(RealWorld)
        V.setConfiguration(RealWorld)
        
        S.step(q, tau, ry.ControlMode.position)

In [ ]:
# Grab and Threow the ball
wait(50)
grab()
liftandthrow()
wait(150)

# Move to desired position
IK = RealWorld.komo_IK(False)

# Get object position
objpos = RealWorld.frame("obj0").getPosition()
objpos[2] = 0.65
    
# Position difference between paddle centre and ball position
IK.addObjective([1], ry.FS.position, ["paddle"], ry.OT.sos, [1e2], target=objpos)
# Z axis is parallel
IK.addObjective([], ry.FS.vectorZ, ["paddle"], ry.OT.eq, [1e2], target = [0.,0.,-1])
# Qitslef
IK.addObjective([], feature=ry.FS.qItself, type=ry.OT.sos, order=1);
  
IK.optimize(True)

RealWorld.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(RealWorld) #to update your model display

In [ ]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.01)
    
    #grab sensor readings from the simulation
    q = S.get_q()
    [distarr,Jdist] = RealWorld.evalFeature(ry.FS.positionDiff, ["paddle", "obj0"])
    dist = np.linalg.norm(distarr)
    print (dist)

    # Get object position
    objpos = RealWorld.frame("obj0").getPosition()
    #objpos[2] = 0.65

    # Move to desired position
    #komo = RealWorld.komo_IK(False)
    # we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
    komo = RealWorld.komo_path(1.,1, 0.045, True)

    # addObjective(times, featureSymbol, frameNames, objectiveType, scale, target, order)
    # To prevent collisions
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
    # Position difference between paddle centre and ball position
    komo.addObjective([1], ry.FS.position, ["paddle"], ry.OT.sos, [1e2], target=objpos)
    # Z axis is parallel
    komo.addObjective([], ry.FS.vectorZ, ["paddle"], ry.OT.eq, [1e2], target = [0.,0.,-1])
    # Qitslef
    komo.addObjective([], feature=ry.FS.qItself, type=ry.OT.sos, order=1);

    komo.optimize(True)

    RealWorld.setFrameState(komo.getConfiguration(0))
    # Get joint state
    q = RealWorld.getJointState()

    # updating the display
    V.recopyMeshes(RealWorld)
    V.setConfiguration(RealWorld)

    if(dist <= 0):
        break
        
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    S.step(q, tau, ry.ControlMode.position)
#   S.step([], tau, ry.ControlMode.none)
